# CellPLATO | Cell Plasticity Analysis Tool

Step 1: Fill in the config file!

Also, make sure your data is set up in the following two tiered format:

        Master
            ├── Condition 1
            │   ├── Replicate 1
            |   |       ├── tracks.h5
            │   ├── Replicate 2
            |   |       ├── tracks.h5            
            │   └── Replicate 3
            |           └── tracks.h5            
            │  
            └── Condition 2,
                ├── Replicate 1
                |       ├── tracks.h5
                ├── Replicate 2
                |       ├── tracks.h5            
                └── Replicate 3
                        └── tracks.h5    

<div class="alert alert-block alert-danger">
Set your kernel to 'cellPLATO' before continuing
</div>

<div class="alert alert-block alert-success">
<h2>1. Start by importing packages for cellPLATO</h1>
</div>

This includes cellPLATO itself, and all of the modules you will need

* Import these packages, checking that you have them
* We're also importing a lot of the modules in cellPLATO, if this cell runs successfully, you are good to go!

In [ ]:
import cellPLATO as cp

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import imageio

import plotly
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import seaborn as sns
from pandas.plotting import scatter_matrix
from matplotlib import pyplot
import matplotlib.cm as cm
import plotly.graph_objects as go
import plotly.express as px

OVERWRITE_DATAFRAMES = True

In [ ]:
import matplotlib
#get matplotlib version
print('matplotlib: {}'.format(matplotlib.__version__))

# Import your experiment list

Check that the list generated in the next cell contains your conditions and replicates

In [ ]:
# Get the experiment list from the experiments listed in the config 
exp_list = cp.populate_experiment_list()
display(exp_list)
print(cp.SAVED_DATA_PATH)

<div class="alert alert-block alert-success">
<h2>2. Measurements of morphology and migration</h1>
</div>

### This cell does migration and morphology measurements for all of the cells at each timepoint 

In [ ]:
# Load, process and combine the dataframes (including segmentation and migration calculations)
comb_df = cp.combine_dataframes(exp_list)

comb_df, new_factors = cp.measurement_pipeline(comb_df, mixed=cp.MIXED_SCALING, factors_to_timeaverage = cp.ALL_FACTORS) 
display(new_factors)

# Returns a filtered dataframe, while also adding included column to comb_df
comb_df, filt_counts = cp.apply_filters(comb_df)

# Process a time-averaged DataFrame
tavg_df = cp.time_average(comb_df)

### Optional: make plots for a chosen factor

In [ ]:
## Factor list: 
all_factors =  ['area',
                'bbox_area',
                'eccentricity',
                'equivalent_diameter',
                'extent',
                'filled_area',
                'major_axis_length',
                'minor_axis_length',
                'orientation',
                'perimeter',
                'solidity',
                'euclidean_dist',     
                'cumulative_length', 
                'speed',
                'orientedness', 
                'directedness',
                'turn_angle',
                'endpoint_dir_ratio',
                'dir_autocorr',
                'outreach_ratio',
                'MSD',                
                'max_dist',           
                'glob_turn_deg',
                'arrest_coefficient'
                'aspect',
                'rip_p',
                'rip_K',
                'rip_L']

In [ ]:
chosen_factor='area'

In [ ]:
# This is a timeplot of the chosen factor (mean, 95% CI) for each condition

f=cp.multi_condition_timeplot(comb_df, factor = chosen_factor)
f.show()

In [ ]:
# This calculated the effect size for each condition relative to the control defined in the config

cp.timeplots_of_differences(comb_df_filt,factor=chosen_factor)

In [ ]:
# This makes a comparative plot of difference based on the time averaged dataframe

f=cp.plots_of_differences_sns(tavg_df,factor=chosen_factor)
f.show()

And then gives you the option to save the dataframes into your automatically created 'saved data' folder

In [ ]:
OVERWRITE_DATAFRAMES = True

if OVERWRITE_DATAFRAMES:
    comb_df.to_csv(cp.SAVED_DATA_PATH + 'comb_df.csv', index=False)
    tavg_df.to_csv(cp.SAVED_DATA_PATH + 'tavg_df.csv', index=False)

### Optional: load in precreated dataframes

In [ ]:
comb_df = pd.read_csv(cp.SAVED_DATA_PATH + 'comb_df.csv')
tavg_df = pd.read_csv(cp.SAVED_DATA_PATH + 'tavg_df.csv')

### Optional: do filtering on the data (on top of what has been stated in the config file)

In [ ]:
# User-defined filters in dict {factor:(min, max)}

data_filters = {
#   "speed": (10, 100),
  "ntpts": (8, 60),
#    "frame": (0, 450), # Warning: range will change if self-normalized
#   "ntpts": (12,1800)
}

# Returns a filtered dataframe, while also adding included column to comb_df
filt_df, filt_counts = cp.apply_filters(comb_df,how='any', filter_dict=data_filters)

fig = cp.visualize_filtering(filt_df, filt_counts)

# Plot all metrics

This cell makes comparative plots for every single metric and saves them in your output folder

* Plots of difference
* Timeplots of difference
* Marginal xy plots
* Simple bar plots
* Superplots - useful for comparing between replicates

<div class="alert alert-block alert-danger">
Check that you are happy with your extra filtering before continuing
Run the next cell on the filtered dataframe or the unfiltered dataframe once you are ready
</div>

In [ ]:
# Outputs plots of all metrics for all factors
cp.comparative_visualization_pipeline(comb_df, num_factors=all_factors) 

<div class="alert alert-block alert-success">
<h2>3. Definition of single timepoint behavioural clusters using UMAP and HDBSCAN</h1>
</div>

### Optional: for new datasets perform correlation analysis to understand which factors correlate to one another

This may aid in choosing the most important factors, aiding clustering

In [ ]:
df_in = comb_df
cp.correlation_matrix_heatmap(df_in, factors = cp.ALL_FACTORS)

### Optional: use variance thresholder for further insight

In [ ]:
chosen_dr_factors = cp.variance_threshold(comb_df, threshold_value=0.03)
chosen_dr_factors

### Optional: define a new list of dr_factors to use for UMAP

In [ ]:
REGIONPROPS_LIST = ['area',
                    'bbox_area',
                    'eccentricity',
                    'equivalent_diameter',
                    'extent',
                    'filled_area',
                    'major_axis_length',
                    'minor_axis_length',
                    'orientation',
                    'perimeter',
                     'solidity'
                     ]

MIG_FACTORS = ['euclidean_dist',     
                'cumulative_length', 
                'speed',
                'orientedness', 
                'directedness',
                'turn_angle',
                'endpoint_dir_ratio',
                'dir_autocorr',
                'outreach_ratio',
                'MSD',                
                'max_dist',           
                'glob_turn_deg',
                'arrest_coefficient']

ADDITIONAL_FACTORS = ['aspect', 'rip_L'] # 'rip_p', 'rip_K', 


DR_FACTORS = REGIONPROPS_LIST + MIG_FACTORS + ADDITIONAL_FACTORS

### Optional: Take a sample of cells from each Replicate_ID

In [ ]:
import random
uniq_ids = comb_df.groupby('Replicate_ID')['uniq_id'].unique().tolist()
# Create a new list to store the sampled unique IDs
sampled_uniq_ids = []
# Iterate over each replicate ID
for i, replicate_id in enumerate(uniq_ids):
    total_sample_size_counter.append(total_sample_size)
    sample_size=len(replicate_id) // 2
    sampled_ids = random.sample(replicate_id.tolist(), sample_size)
    sampled_uniq_ids.extend(sampled_ids)

print(len(sampled_uniq_ids))
# print(sampled_uniq_ids.shape())



In [ ]:
import random
uniq_ids = comb_df.groupby('Replicate_ID')['uniq_id'].unique().tolist()

# Create a new list to store the sampled unique IDs
sampled_uniq_ids = []

for condition in comb_df['Condition'].unique():
    condition_df = comb_df[comb_df['Condition'] == condition]
    uniq_ids = condition_df.groupby('Replicate_ID')['uniq_id'].unique().tolist()
    
    for replicate_id in condition_df['Replicate_ID'].unique():

        replicate_df = condition_df[condition_df['Replicate_ID'] == replicate_id]
        rep_uniq_ids = replicate_df['uniq_id'].unique().tolist()
        sample_size = len(rep_uniq_ids) // 2
        sampled_ids = random.sample(rep_uniq_ids, sample_size)
        sampled_uniq_ids.extend(sampled_ids)



## Perform UMAP and cluster analysis

### First, do UMAP, save the new df and plot the UMAP

Well separated clusters depend mostly on 1. the input factors and 2. the umap_nn setting

You can change both, depending on the nature of your data, in order to achieve a reasonable level of separation of clusters

In [ ]:
###### User alterable parameters ######
tsne_perp=150
umap_nn = 50 #umap nearest neighbours
min_dist = 0.0 #umap minimum distance (usually keep this at 0 or very low)
n_components = 3 # number of umap dimensions to calculate
#######################################


# for umap_nn in [40,50, 60, 70, 80, 90, 100]:

dr_df = cp.dr_pipeline_multiUMAPandTSNE(comb_df, 
                    dr_factors=DR_FACTORS,
                    n_components = n_components,
                    umap_nn=umap_nn,
                    min_dist= min_dist,
                    scalingmethod = 'choice',
                    do_tsne=False) # A number of scaling methods are available: 'choice', 'minmax', 'standard', 'robust', 'normalize', 'quantile', 'maxabs', 'yeo-johnson', 'box-cox'

dr_df.to_csv(cp.SAVED_DATA_PATH + 'dr_df_'+ str(umap_nn) + '.csv', index=False) # Saves the df

cp.plot_3D_scatter(dr_df, 'UMAP1', 'UMAP2', 'UMAP3', colorby='condition', ticks=False, identifier='dr_df_' + str(umap_nn), dotsize = 0.01, alpha=0.1, markerscale = 100) #color = label or condition  

In [ ]:
# save the dr_df
dr_df.to_csv(cp.SAVED_DATA_PATH + 'dr_df.csv', index=False)

In [ ]:
##### User adjustable parameters #####
min_cluster_size = 800
min_samples = 300 #300 #500
cluster_by = 'UMAPNDIM' # UMAPNDIM = default, clusters on UMAPs. NDIM = alternate, clusters on all dimensions
metric = 'euclidean' # See https://hdbscan.readthedocs.io/en/latest/api.html#hdbscan.HDBSCAN for options
#######################################

lab_dr_df, exemplar_df=cp.hdbscan_clustering(dr_df, min_cluster_size=min_cluster_size, min_samples=min_samples, cluster_by=cluster_by,  metric=metric)

lab_dr_df.name='lab_dr_df'
name = lab_dr_df.name

lab_dr_df.to_csv(cp.SAVED_DATA_PATH + 'lab_dr_df' + str(umap_nn)+str(min_samples)+'.csv', index=False)
exemplar_df.to_csv(cp.SAVED_DATA_PATH + 'exemplar_df' + str(umap_nn)+str(min_samples)+'.csv', index=False)

cp.plot_3D_scatter(lab_dr_df, 'UMAP1', 'UMAP2', 'UMAP3', colorby='label', ticks=False, identifier=name + 'thisone'+str(umap_nn)+str(min_samples),dotsize = 0.01, alpha=0.1, markerscale = 100) #color = label or condition   


## Then plot the 'fingerprint' plot of percentage in each cluster per condition

In [ ]:
# This is the new combo
cluster_purity_df = cp.purity_pointsinclusterspercondition(lab_dr_df) 
display(cluster_purity_df)
f = cp.purityplot_percentcluspercondition(lab_dr_df, cluster_purity_df) 

### Optional: explore the clusters with interactive 3D plot

In [ ]:
cp.interactive_plot_3D_UMAP(df=lab_dr_df,colorby = 'Condition_shortlabel', symbolby = 'Condition_shortlabel', what = ' AllTimeUMAPwithclusters') # TavgUMAPwithclusters

### Optional: all other conditions colored grey, chosen condition in color

In [ ]:
df=lab_dr_df

condlist = df['Condition_shortlabel'].unique().tolist() #get unique list of conditions from df
print(condlist) # show the condition list
# chosen_condition = '' #specify a chosen condition from the list
chosen_condition = condlist[0] # or choose the first one
print(chosen_condition)

cp.interactive_plot_3D_UMAP_chosen_condition(df, chosen_condition, opacity_grey=0.01, marker_size_all=2,) #change opacity and marker size to suit the data

### Optional: make UMAP plots colored by metric contributors - the more intense the color, the higher the contribution the metric to a cluster

In [ ]:
# First one colors per metric
cp.plot_UMAP_subplots_coloredbymetricsorconditions(df_in=lab_dr_df, x= 'UMAP1', y= 'UMAP2', z = 'UMAP3', n_cols = 5, ticks=False, metrics = cp.ALL_FACTORS, scalingmethod='choice',
                                                   identifier='inferno', colormap='inferno', coloredbycondition = False, samplethedf = False)
#second one colors per condition
# cp.plot_UMAP_subplots_coloredbymetricsorconditions(df_in=tptlabel_dr_df, x= 'UMAP1', y= 'UMAP2', z = 'UMAP3', n_cols = 5, ticks=False, metrics = cp.ALL_FACTORS, scalingmethod='choice',
#                                                    identifier='inferno', colormap='inferno', coloredbycondition = True, samplethedf = False)

### Perform UMAP then HDBSCAN on the tavg_df (deprecated)

This step used to add tavg_label to the dataframe. Its deprecated and can be skipped.

In [ ]:
tsne_perp=150
umap_nn = 20#4#60
min_dist = 0.0 #0.15 
n_components = 3

tavg_dr_df = cp.dr_pipeline_multiUMAPandTSNE(tavg_df, 
                    dr_factors=newnew_DR_FACTORS,# new_DR_FACTORS # DR_FACTORS #only_tmeans # cp.DR_FACTORS
                    n_components = n_components,
                    umap_nn=umap_nn,
                    min_dist= min_dist,
                    scalingmethod = 'choice',) # log2minmax # powertransformer #minmax

lab_tavg_dr_df, exemplar_tavg_df=cp.hdbscan_clustering(tavg_dr_df, min_cluster_size=50,min_samples=50,cluster_by='UMAPNDIM',  metric='euclidean', plot=False) # 

#Run this function to put the labels into the lab_tavg_lab_dr_df. Slow function. Can update search by uniq_id alone...

lab_tavg_lab_dr_df=cp.add_tavglabel_todf(lab_dr_df_ACTUAL, lab_tavg_dr_df)
lab_tavg_lab_dr_df.to_csv(cp.SAVED_DATA_PATH + 'lab_tavg_lab_dr_df.csv', index=False)
 

Instead, just make lab_tavg_dr_df by adding random labels to a new column called tavg_label

In [ ]:
lab_dr_df['tavg_label'] = np.random.randint(0, 5, lab_dr_df.shape[0])
lab_tavg_lab_dr_df = lab_dr_df
# lab_tavg_lab_dr_df.to_csv(cp.SAVED_DATA_PATH + 'lab_tavg_lab_dr_df.csv', index=False)


<div class="alert alert-block alert-danger">
Save your dataframes so you can come back to this step if necessary
</div>

In [ ]:
OVERWRITE_DATAFRAMES = True

if OVERWRITE_DATAFRAMES:
    tavg_dr_df.to_csv(cp.SAVED_DATA_PATH + 'tavg_dr_df.csv', index=False)
    lab_tavg_dr_df.to_csv(cp.SAVED_DATA_PATH + 'lab_tavg_dr_df.csv', index=False)
    exemplar_tavg_df.to_csv(cp.SAVED_DATA_PATH + 'exemplar_tavg_df.csv', index=False)

## Quantify the plasticity 

This cell counts the number of single timepoint cluster switches, creating new dataframe tptlabel_dr_df

In [ ]:
tptlabel_dr_df = cp.count_cluster_changes_with_tavg(lab_tavg_lab_dr_df)
tptlabel_dr_df.to_csv(cp.SAVED_DATA_PATH + 'tptlabel_dr_df.csv', index=False)

### Plots of plasticity

In [ ]:
tptlabel_dr_df = pd.read_csv(cp.SAVED_DATA_PATH + 'tptlabel_dr_df.csv')

In [ ]:
df=tptlabel_dr_df
cp.plot_plasticity_changes(df, identifier='\_allcells', maxy=4) 

In [ ]:
df=tptlabel_dr_df
cp.plot_plasticity_countplots(df, identifier='_allcells')

In [ ]:
df=tptlabel_dr_df
cp.plot_cumulative_plasticity_changes_main(df, identifier='\_allcells', miny=None, maxy=None, t_window_multiplier = cp.T_WINDOW_MULTIPLIER, plotallcells = False)

# De-abstractify the clusters!
This set of functions makes plots to describe the nature of cells from each cluster

### First, choose a number of exemplar cells to pick out from the exemplar cell list to display

In [ ]:
# Choose a number of exemplars to display for each cluster
n=5
exemplar_df = exemplar_df.groupby('label').apply(lambda x: x.sample(min(n,len(x)))).reset_index(drop=True)

### Show exemplar cells overlaid onto raw data

Note - you must have the raw data in the folder with the masks

In [ ]:
cp.make_raw_cell_png_overlaidwith_behaviourcontourandtrack(tptlabel_dr_df, exemplar_df, LUTlow=5, LUThi=140, XYRange = 200, invert=True)

### Optional: make a full de-abstractification

The following functions 
1. Rank features based on their contribution to the formation of each cluster
2. compute averages of each feature per cluster and print a table
3. Makes per cell plots over time of a) top contributing features, b) cluster switches and c) contour maps

In [ ]:
size=220 #

df= tptlabel_dr_df #from the all analysis part
exemp_df=exemplar_df #from the cluster analysis part.

top_dictionary, contributions_df_singletpoints, scaled_df=cp.contribution_to_clusters(df_in=tptlabel_dr_df,  howmanyfactors=3, dr_factors= DR_factors) #BEFORE disambiguate_tavg(), then: lab_tavg_dr_df BEFORE disambiguate_timepoint(), then: #tptlabel_dr_df 
cp.plot_cluster_averages(top_dictionary, df, scaled_df)
result_df = cp.create_cluster_averages_table(top_dictionary, df, scaled_df)
cp.disambiguate_timepoint(df, exemp_df, scaled_df, top_dictionary=top_dictionary, XYRange=size,boxoff=True, trajectory = False) 

In [ ]:

tptlabel_dr_df = pd.read_csv(cp.SAVED_DATA_PATH + 'tptlabel_dr_df.csv')
exemplar_df = pd.read_csv(cp.SAVED_DATA_PATH + 'exemplar_df.csv')

Optional: plots plasticity changes in a subset of exemplar cells over time

In [ ]:
#### User inputs ####
whole_df = tptlabel_dr_df
exemplar_df = exemplar_df
numberofdesiredtimepoints = int(whole_df['ntpts'].mean())
numberofcellspercluster = 40
num_clusters_whole_dataset = len(whole_df['label'].unique())
# override = int((numberofcellspercluster*num_clusters_whole_dataset)*0.7)
#####################

exemplar_df_filt, exemplar_cell_tracks_df = cp.filter_exemplars(whole_df=whole_df, exemplar_df = exemplar_df, numberofdesiredtimepoints = numberofdesiredtimepoints, 
                                                                    numberofcellspercluster = numberofcellspercluster, override = override)

In [ ]:
df=exemplar_cell_tracks_df
cp.plot_cumulative_plasticity_changes_main(df, identifier='\_exemplars_only_3_df__', miny=None, maxy=None, t_window_multiplier = 1, plotallcells = False)

### Plot any factor as small multiples from the exemplars

Gives a line plot of any feature desired from the exemplars

In [ ]:
df = exemplar_cell_tracks_df
whichcolumntoplot = 'label'

cp.plot_small_multiples(df, whichcolumntoplot)

<div class="alert alert-block alert-success">
<h2>4. Trajectory measurement: Damerau-Levenshtein</h1>
</div>

This part measures the similarity of sequences of single timepoint cluster IDs, in order to create trajectories of behaviour that can be compared between conditions

In [ ]:
tptlabel_dr_df = pd.read_csv(cp.SAVED_DATA_PATH + 'tptlabel_dr_df.csv')

### First filter the tptlabel_dr_df to include only a subset of data of similar timescale

In [ ]:
low = 180
high = 220

tptlabel_dr_df_filt = tptlabel_dr_df[tptlabel_dr_df['ntpts'].between(low, high)]
print(f'You are sampling {len(tptlabel_dr_df_filt.uniq_id.unique())} cells')

### Verify that the filtered data reflects the total data

In [ ]:
factorchoice = 'speed'

In [ ]:
# Makes timeplots of the unfiltered and filtered data

f=cp.multi_condition_timeplot(tptlabel_dr_df, factorchoice)
f.show()
f=cp.multi_condition_timeplot(tptlabel_dr_df_filt, factorchoice)
f.show()

In [ ]:
# Plot of difference of the unfiltered and filtered data
f = cp.plots_of_differences_sns(tavg_df, factorchoice)
f.show()

In [ ]:
tavg_trajectory_df = cp.time_average(tptlabel_dr_df_filt)
f = cp.plots_of_differences_sns(tavg_trajectory_df, factorchoice)
f.show()

### Perform Damerau-Levenshtein analysis

In [ ]:
df = tptlabel_dr_df_filt
distance_matrix_dameraulev = cp.calculate_edit_distances(df,distancemetric = 'dameraulev', print_interval=10000) #fastdtw # dameraulev # mongeelkan
print(distance_matrix_dameraulev.shape)

### Perform a UMAP/HDBSCAN parameter sweep, and select plots

In [ ]:
'''Sweep through a number of UMAP/HDBSCAN settings to ensure good clustering '''

df = tptlabel_dr_df_filt
for n_neighbors in [8, 10, 12]:
    for min_samples in [5,8,10, 15, 30, 40]:
        for min_cluster_size in [5,8,10, 15, 30, 40]:
            print(f'min_samples = {min_samples}')
            print(f'min_cluster_size = {min_cluster_size}')
            print(f'n_neighbors = {n_neighbors}')
            tptlabel_dr_df_filt_clusteredtrajectories = cp.cluster_sequences(df, distance_matrix_dameraulev,
             do_umap=True, eps=0.1, min_samples=min_samples, min_cluster_size=min_cluster_size, n_neighbors=n_neighbors)

In [ ]:
'''Chosen UMAP and HDBSCAN parameters'''

min_samples = 10
min_cluster_size = 5
n_neighbors = 8

df = tptlabel_dr_df_filt

print(f'min_samples = {min_samples}')
print(f'min_cluster_size = {min_cluster_size}')
print(f'n_neighbors = {n_neighbors}')
tptlabel_dr_df_filt_clusteredtrajectories = cp.cluster_sequences(df, distance_matrix_dameraulev,
 do_umap=True, eps=0.1, min_samples=min_samples, min_cluster_size=min_cluster_size, n_neighbors=n_neighbors)

In [ ]:
# remove the -1 TRAJECTORY cluster if required
# tptlabel_dr_df_filt_clusteredtrajectories = tptlabel_dr_df_filt_clusteredtrajectories[tptlabel_dr_df_filt_clusteredtrajectories['trajectory_id'] != -1]

Get the fingerprint plot of trajectories

In [ ]:
df = tptlabel_dr_df_filt_clusteredtrajectories

cluster_purity_df = cp.purity_pointsinclusterspercondition(df, cluster_label='trajectory_id') 
f = cp.purityplot_percentcluspercondition(df, cluster_purity_df, cluster_label='trajectory_id', dotsize = 30) 

 ### Disambiguate the trajectory clustered cells:
 1) Make an exemplar_df_trajectories containing example rows
 2) Get the full tracks from those rows and make exemplar_df_trajectories_fulltrack
 2) Disambiguate with exemplar_df_trajectories
 3) Plot multiples with exemplar_df_trajectories_fulltrack

In [ ]:
df = tptlabel_dr_df_filt_clusteredtrajectories
exemplar_df_trajectories, exemplar_df_trajectories_fulltrack  = cp.make_exemplar_df_basedon_trajectories(df, cells_per_traj=30)

In [ ]:
# full_tracks_df = pd.read_csv(cp.SAVED_DATA_PATH + 'full_tracks_df.csv')
df = exemplar_df_trajectories_fulltrack
cp.plot_trajectories(df=exemplar_df_trajectories_fulltrack, global_y=True, global_x=True)

### Optional: full deabstractification of trajectory clusters

In [ ]:
size=300 #


df= tptlabel_dr_df_filt_clusteredtrajectories 
exemp_df=exemplar_df_trajectories 

top_dictionary, contributions_df_singletpoints, scaled_df=cp.contribution_to_clusters(df_in=tptlabel_dr_df,  howmanyfactors=2, dr_factors= DR_FACTORS) #BEFORE disambiguate_tavg(), then: lab_tavg_dr_df BEFORE disambiguate_timepoint(), then: #tptlabel_dr_df 
cp.plot_cluster_averages(top_dictionary, df, scaled_df)
result_df = cp.create_cluster_averages_table(top_dictionary, df, scaled_df)
cp.disambiguate_timepoint(df, exemp_df, scaled_df, top_dictionary=top_dictionary, XYRange=size,boxoff=True, trajectory = True) 

# Percent fingerprint plot for cluster IDs per TRAJECTORY

In [ ]:
df = tptlabel_dr_df_filt_clusteredtrajectories
cp.fingerprintplot_clusters_per_trajectory(df)

# Plasticity of cells per trajectory

In [ ]:
df=tptlabel_dr_df_filt_clusteredtrajectories
cp.plasticity_per_trajectory(df)

In [ ]:

def plasticity_per_condition(df):

    # Set the style and color palette
    sns.set_style("whitegrid")
    max_values = df.groupby(['Condition_shortlabel', 'uniq_id'])['cum_n_changes'].max().reset_index()
    display(max_values)
    median_values = max_values.groupby(['Condition_shortlabel'])['cum_n_changes'].median().reset_index()

    colors = []
    cmap = cm.get_cmap(cp.CONDITION_CMAP)
    numcolors=len(df['Condition_shortlabel'].unique())
    for i in range(numcolors):
        colors.append(cmap(i))    

    plt.figure(figsize=(15,10))  # Adjust the size of the plot

    barplot = sns.barplot(x='Condition_shortlabel', y='cum_n_changes', data=median_values, color='grey', alpha = 0.5)
    sns.violinplot(x='Condition_shortlabel', y='cum_n_changes', data=max_values, palette=colors)

    # Add the median values to the plot
    for i, bar in enumerate(barplot.patches):
        barplot.annotate(format(median_values['cum_n_changes'].values[i], '.2f'), 
                        (bar.get_x() + bar.get_width() / 2, bar.get_height()), 
                        ha = 'center', va = 'center', 
                        xytext = (0, -10),  
                        textcoords = 'offset points',
                        fontsize = 35)
    # Increase the font size
    plt.xticks(fontsize=35)
    plt.yticks(fontsize=35)
    # Increase font size of x and y labels
    plt.xlabel('', fontsize=35)
    plt.ylabel('Cumulative cluster switches', fontsize=35)
    # set the ylim
    plt.ylim(-10, 150)
    # save it in the trajectory disambig folder
    # plt.savefig(TRAJECTORY_DISAMBIG_DIR + 'plasticity_per_trajectory.png', dpi=300)

    plt.show()
    
    return

In [ ]:
plasticity_per_condition(tptlabel_dr_df_filt_clusteredtrajectories)

In [ ]:
df=tptlabel_dr_df_filt_clusteredtrajectories
# all='\_allcells'
cp.plot_plasticity_changes_trajectories(df, identifier='\_allcells', maxy=9 , t_window_multiplier = 1) #problem with NaNs in the data

In [ ]:
#save all newly created dataframes
tptlabel_dr_df_filt_clusteredtrajectories.to_csv(cp.SAVED_DATA_PATH + 'tptlabel_dr_df_filt_clusteredtrajectories.csv', index=False)
exemplar_df_trajectories.to_csv(cp.SAVED_DATA_PATH + 'exemplar_df_trajectories.csv', index=False)

In [ ]:
# read in exemplar_df_trajectories
exemplar_df_trajectories = pd.read_csv(cp.SAVED_DATA_PATH + 'exemplar_df_trajectories.csv')

# Animations of trajectories

In [ ]:
# read in tptlabel_dr_df_filt_clusteredtrajectories
tptlabel_dr_df_filt_clusteredtrajectories = pd.read_csv(cp.SAVED_DATA_PATH + 'tptlabel_dr_df_filt_clusteredtrajectories.csv')

In [ ]:
df = tptlabel_dr_df_filt_clusteredtrajectories
cp.make_trajectory_animations(df, exemplar_df_trajectories, number_of_trajectories=10, colormode='trajectory') # singlecluster, cluster, trajectory

Select a number of example cells from each trajectory ID to map back on to the data and display as stacks of PNGs

In [ ]:
number_of_trajectories = 30 # Select a number of trajectories to plot

df = tptlabel_dr_df_filt_clusteredtrajectories

trajectory_ids = df['trajectory_id'].unique()

uniq_id_choices_list = []

for trajectory_id_choice in trajectory_ids:
    # for each trajectory_id, get a list of possible uniq_ids from the df
    uniq_id_choices = tptlabel_dr_df_filt_clusteredtrajectories[tptlabel_dr_df_filt_clusteredtrajectories['trajectory_id']==trajectory_id_choice]['uniq_id'].values
    # Make sure each once is unique in that list
    uniq_id_choices = np.unique(uniq_id_choices)
    # choose a number of random uniq_ids from that list based on number_of_trajectories
    uniq_id_choices = np.random.choice(uniq_id_choices, number_of_trajectories)
    # append each choice to a list
    uniq_id_choices_list.append(uniq_id_choices)
# flatten the list
chosen_uniq_ids = [item for sublist in uniq_id_choices_list for item in sublist]
    
print(f'There are {len(chosen_uniq_ids)} cells')

In [ ]:
#Filter the df to only include the chosen uniq_ids
subset_trajectories_df = tptlabel_dr_df_filt_clusteredtrajectories[tptlabel_dr_df_filt_clusteredtrajectories['uniq_id'].isin(chosen_uniq_ids)]

This function makes png image sequences of cells belonging to each trajectory and saves them for visualization. By changing the colormode from trajectory to cluster, you can color the cells by single timepoint cluster or trajectory cluster

In [ ]:
df = subset_trajectories_df
cp.make_png_behaviour_trajectories(df,chosen_uniq_ids,XYRange = 300, follow_cell = False, invert=False, colormode = 'trajectory', snapshot=False)

In [ ]:
df = subset_trajectories_df
cp.make_png_behaviour_trajectories(df,chosen_uniq_ids,XYRange = 300, follow_cell = False, invert=False, colormode = 'cluster')

This makes a png stack of the raw images to match the above contour maps

In [ ]:
df = subset_trajectories_df
cp.make_raw_cell_pngstacks(df,chosen_uniq_ids,XYRange = 300, follow_cell=False, invert=False)

STITCHING FUNCTION - its hard to manually visualize all of the trajectories for each cell, so this function allows one to make a big stitched image for visualization

In [ ]:
import os
from PIL import Image

def stitch_and_save_image_sequence_varying_sizes(master_folder, output_folder):
    # Ensure the output folder exists
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    
    subfolders = [os.path.join(master_folder, f) for f in os.listdir(master_folder) if os.path.isdir(os.path.join(master_folder, f))]
    subfolders.sort()  # Ensure consistent order
    
    # Load all images to determine individual sizes
    all_images = {}
    for subfolder in subfolders:
        images = [f for f in os.listdir(subfolder) if f.endswith('.png')]
        images.sort(key=lambda x: int(x.split('_')[-1].split('.')[0]))  # Sorting by time point
        all_images[subfolder] = [Image.open(os.path.join(subfolder, img)) for img in images]
    
    # Determine the number of images (time points) based on the first subfolder (assuming consistent across subfolders)
    num_images = len(all_images[subfolders[0]])

    for i in range(num_images):
        row_heights = []
        max_width = 0
        images_to_stitch = []

        for row in range(0, len(subfolders), 4):
            row_images = []
            row_max_height = 0
            row_width = 0

            for col in range(4):
                idx = row + col
                if idx < len(subfolders):
                    images = all_images[subfolders[idx]]
                    if i < len(images):
                        img = images[i]
                        row_images.append(img)
                        row_max_height = max(row_max_height, img.height)
                        row_width += img.width
            
            max_width = max(max_width, row_width)
            row_heights.append(row_max_height)
            images_to_stitch.append(row_images)

        # Calculate total grid height
        total_height = sum(row_heights)
        final_image = Image.new('RGB', (max_width, total_height))

        y_offset = 0
        for row_idx, row_images in enumerate(images_to_stitch):
            x_offset = 0
            for img in row_images:
                final_image.paste(img, (x_offset, y_offset))
                x_offset += img.width
            y_offset += row_heights[row_idx]

        output_filename = os.path.join(output_folder, f"stitched_{i+1:04d}.png")
        final_image.save(output_filename)

# Example usage
# stitch_and_save_image_sequence_varying_sizes('path/to/master_folder', 'path/to/output_folder')


In [ ]:
import os
from PIL import Image

def stitch_and_save_image_sequence_varying_sizes(master_folder, output_folder, reduce_size=False):
    # Ensure the output folder exists
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    
    subfolders = [os.path.join(master_folder, f) for f in os.listdir(master_folder) if os.path.isdir(os.path.join(master_folder, f))]
    subfolders.sort()  # Ensure consistent order
    
    # Load all images to determine individual sizes
    all_images = {}
    for subfolder in subfolders:
        images = [f for f in os.listdir(subfolder) if f.endswith('.png')]
        images.sort(key=lambda x: int(x.split('_')[-1].split('.')[0]))  # Sorting by time point
        all_images[subfolder] = [Image.open(os.path.join(subfolder, img)) for img in images]
    
    # Determine the number of images (time points) based on the first subfolder
    num_images = len(all_images[subfolders[0]])

    for i in range(num_images):
        row_heights = []
        max_width = 0
        images_to_stitch = []

        for row in range(0, len(subfolders), 4):
            row_images = []
            row_max_height = 0
            row_width = 0

            for col in range(4):
                idx = row + col
                if idx < len(subfolders):
                    images = all_images[subfolders[idx]]
                    if i < len(images):
                        img = images[i]
                        row_images.append(img)
                        row_max_height = max(row_max_height, img.height)
                        row_width += img.width
            
            max_width = max(max_width, row_width)
            row_heights.append(row_max_height)
            images_to_stitch.append(row_images)

        # Calculate total grid height
        total_height = sum(row_heights)
        final_image = Image.new('RGB', (max_width, total_height))

        y_offset = 0
        for row_idx, row_images in enumerate(images_to_stitch):
            x_offset = 0
            for img in row_images:
                final_image.paste(img, (x_offset, y_offset))
                x_offset += img.width
            y_offset += row_heights[row_idx]

        if reduce_size:
            # Reduce the size to a quarter (half the width and half the height)
            final_image = final_image.resize((final_image.width // 2, final_image.height // 2))

        output_filename = os.path.join(output_folder, f"stitched_{i+1:04d}.png")
        final_image.save(output_filename)

# Example usage
# stitch_and_save_image_sequence_varying_sizes('path/to/master_folder', 'path/to/output_folder', reduce_size=True)


In [ ]:
master='D://Michael_Shannon/FUNCTIONTEST/'
output='D://Michael_Shannon/FUNCTIONTEST_output3/'



# Example usage
stitch_and_save_image_sequence_varying_sizes(master, output)

In [ ]:
# folder_path = 'D://Michael_Shannon/CELLPLATO_MASTER/COMBO_Donor2_Donor3_Donor7_OUTPUT/Donor2_3_and7_renamedto4/2024-03-20_21-55-01-520669/plots/Clustering/Trajectory_Cluster_Disambiguation/'

# folder_path='D:\Michael_Shannon\Trajectory_Cluster_Disambiguation'

# folder_path='D:\Michael_Shannon\CELLPLATO_MASTER\JCS_ReviewersComments_2-2024\edited\VERSION_DONOR1_DONORS2_and_4\ABCD_VERSION1\Trajectory_Cluster_DisambiguationABCD\TRAJECTORY'
# folder_path='D:\Michael_Shannon\CELLPLATO_MASTER\JCS_ReviewersComments_2-2024\edited\VERSION_DONOR1_DONORS2_and_4\ABCD_VERSION2'
folder_path='D:\Michael_Shannon\CELLPLATO_MASTER\JCS_ReviewersComments_2-2024\edited\VERSION_DONOR1_DONORS2_and_4\ABCDVERSION3_150\Trajectory_Cluster_Disambiguation'

for foldername in os.listdir(folder_path):
    print(foldername)
    master = os.path.join(folder_path, foldername)
    print(master)

    # add the suffix _output onto the end of master
    output = master + '_output'
    print(output)
    # clear the folder_path variable before the next loop




    # Example usage
    stitch_and_save_image_sequence_varying_sizes(master, output, reduce_size=True)

In [ ]:
import os
# folder_path = 'D://Michael_Shannon/CELLPLATO_MASTER/COMBO_Donor2_Donor3_Donor7_OUTPUT/Donor2_3_and7_renamedto4/2024-03-20_21-55-01-520669/plots/Clustering/Trajectory_Cluster_Disambiguation/'

# folder_path='D:\Michael_Shannon\Trajectory_Cluster_Disambiguation'

# folder_path='D:\Michael_Shannon\CELLPLATO_MASTER\JCS_ReviewersComments_2-2024\edited\VERSION_DONOR1_DONORS2_and_4\ABCD_VERSION1\Trajectory_Cluster_DisambiguationABCD\TRAJECTORY'
# folder_path='D:\Michael_Shannon\CELLPLATO_MASTER\JCS_ReviewersComments_2-2024\edited\VERSION_DONOR1_DONORS2_and_4\ABCD_VERSION2'
folder_path = 'D:\Michael_Shannon\CELLPLATO_MASTER\JCS_ReviewersComments_2-2024\edited\VERSION_DONOR1_DONORS2_and_4\ABCDVERSION3_150\Trajectory_Cluster_Disambiguation'

for foldername in os.listdir(folder_path):
    if os.path.isdir(os.path.join(folder_path, foldername)):
        print(foldername)
        master = os.path.join(folder_path, foldername)
        print(master)

        # add the suffix _output onto the end of master
        output = master + '_output'
        print(output)
        # clear the folder_path variable before the next loop

        # Example usage
        stitch_and_save_image_sequence_varying_sizes(master, output, reduce_size=True)


Extra Things if needed

In [ ]:
f=cp.plots_of_differences_donors(tavg_df,factor=chosen_factor)
f.show()